In [1]:
import numpy as np
import pandas as pd

# Function to parse the data from the file
def parse_data(file_path):
    columns = ['Filter', 'MJD', 'Mag', 'MagErr', '3SigMagLim', '0.98SatLim', 'Rate', 'RateErr', 'Ap', 'Frametime', 'Exp', 'Telapse']
    data = []

    # Open and read the file
    with open(file_path, 'r') as f:
        for line in f:
            # Ignore lines starting with '#' (comments)
            if line.startswith("#"):
                continue
            parts = line.split()
            if len(parts) == 12:  # Ensure correct number of columns
                filter_name, mjd, mag, mag_err, sig_lim, sat_lim, rate, rate_err, ap, frame_time, exp, telapse = parts
                # Handle 'NULL' entries as None
                rate = np.nan if rate == 'NULL' else float(rate)
                mjd = float(mjd)
                data.append([filter_name, mjd, rate, rate_err,telapse])

    # Convert the list to a pandas DataFrame
    df = pd.DataFrame(data,columns=['Filter','MJD','Rate','RateErr','Telapse'])  # Only use Filter, MJD, and Rate
    #print(df)
    return df



In [2]:
def group_mjds(df, tolerance=0.15):
    # Get unique MJDs
    unique_mjd = sorted(df['MJD'].unique())
    avgmjd_list=[]
    # Iterate through each unique MJD
    for mjd in unique_mjd:
        # Filter data within 0.15 days of the current MJD
        subset = df[np.abs(df['MJD'] - mjd) <= tolerance ]
        #subset = df[np.abs(df['MJD'] - mjd) <= tolerance or (((df['MJD'] - df['Telapse']/60/60/24/2) < mjd) and ((df['MJD'] + df['Telapse']/60/60/24/2) > mjd))]
        avgmjd=np.mean(subset['MJD'])
        #print(avgmjd)
        avgmjd_list.append(avgmjd)
    mjddf=pd.DataFrame(avgmjd_list)
    #print(mjddf)
    mjd_groups=sorted(mjddf[0].unique())
    #print(mjd_groups)
    return mjd_groups 


In [3]:
# Function to group data by MJD within the 0.15 MJD tolerance
def organize_by_mjd(df, tolerance=0.15):
    # Initialize an empty list to store rows
    result = []

    # Get unique MJDs
    unique_mjd = sorted(df['MJD'].unique())

    avgmjd_list=[]
    # Iterate through each unique MJD
    for mjd in unique_mjd:
        # Filter data within 0.15 days of the current MJD
        subset = df[np.abs(df['MJD'] - mjd) <= tolerance ]
        #subset = df[np.abs(df['MJD'] - mjd) <= tolerance or (((df['MJD'] - df['Telapse']/60/60/24/2) < mjd) and ((df['MJD'] + df['Telapse']/60/60/24/2) > mjd))]
        avgmjd=np.mean(subset['MJD'])
        #print(avgmjd)
        avgmjd_list.append(avgmjd)
    mjddf=pd.DataFrame(avgmjd_list)
    #print(mjddf)
    mjd_groups=sorted(mjddf[0].unique())
    

    
    # Iterate through each unique MJD
    for mjd in mjd_groups:
        # Filter data within 0.15 days of the current MJD
        subset = df[np.abs(df['MJD'] - mjd) <= tolerance]
        #print(subset)
        # Initialize a row with MJD and placeholders for each filter
        row = [mjd, None, None, None, None, None, None, None, None, None, None, None, None]  # MJD, UVW2, UVM2, UVW1, U, B, V
        
        # Fill in the count rates for each filter
        for _, entry in subset.iterrows():
            #print(entry)
            if entry['Filter'] == 'UVW2':
                row[1] = entry['Rate']
                row[2] = entry['RateErr']
            elif entry['Filter'] == 'UVM2':
                row[3] = entry['Rate']
                row[4] = entry['RateErr']
            elif entry['Filter'] == 'UVW1':
                row[5] = entry['Rate']
                row[6] = entry['RateErr']
            elif entry['Filter'] == 'U':
                row[7] = entry['Rate']
                row[8] = entry['RateErr']
            elif entry['Filter'] == 'B':
                row[9] = entry['Rate']
                row[10] = entry['RateErr']
            elif entry['Filter'] == 'V':
                row[11] = entry['Rate']
                row[12] = entry['RateErr']
        
        # Append the row to the result
        result.append(row)

    # Convert result to a numpy array for convenience
    result_array = np.array(result)
    return result_array


In [6]:

# Main function to run the program
def main(file_path):
    # Parse the data from the file
    df = parse_data(file_path)

    mjd_groups=group_mjds(df, tolerance=0.15)
    
    # Organize the data by MJD with the tolerance of 0.15 days
    result_array = organize_by_mjd(df)

    # Convert to a DataFrame for better visualization
    result_df = pd.DataFrame(result_array, columns=['MJD', 'UVW2', 'UVW2err', 'UVM2', 'UVM2err', 'UVW1', 'UVW1err', 'U', 'Uerr', 'B', 'Berr', 'V', 'Verr'])
    
    print(result_df)
    return result_df

# Provide the path to your data file here
#file_path = '../input/SN2007af_uvotB15.1.dat'
result = main(file_path)


              MJD   UVW2 UVW2err   UVM2 UVM2err    UVW1 UVW1err        U  \
0     54161.85323  0.267   0.048  0.032   0.025   1.199   0.097   15.275   
1     54165.06481   1.02   0.108  0.106   0.039   5.258   0.337   60.515   
2    54169.660617  1.279   0.131  0.606   0.084  12.294   0.716  121.711   
3     54170.63372  2.411   0.226  0.867   0.119  12.538   0.713  121.721   
4    54171.535133  2.517   0.205  0.686   0.091  13.576   0.756   126.88   
..            ...    ...     ...    ...     ...     ...     ...      ...   
129  54448.107269 -0.019   0.040 -0.042   0.029    None    None     None   
130  54448.140655  -0.01   0.041  0.051   0.037    None    None     None   
131  54448.207255  0.016   0.042  0.008   0.034    None    None     None   
132  54448.240694  0.016   0.042  0.008   0.034    None    None     None   
133    54448.2742  0.016   0.042  0.008   0.034    None    None     None   

      Uerr        B   Berr       V   Verr  
0    0.797   39.971  1.381  14.393  0.695  

In [7]:
main('../input/SN2007af_uvotB15.1.dat')

              MJD   UVW2 UVW2err   UVM2 UVM2err    UVW1 UVW1err        U  \
0     54161.85323  0.267   0.048  0.032   0.025   1.199   0.097   15.275   
1     54165.06481   1.02   0.108  0.106   0.039   5.258   0.337   60.515   
2    54169.660617  1.279   0.131  0.606   0.084  12.294   0.716  121.711   
3     54170.63372  2.411   0.226  0.867   0.119  12.538   0.713  121.721   
4    54171.535133  2.517   0.205  0.686   0.091  13.576   0.756   126.88   
..            ...    ...     ...    ...     ...     ...     ...      ...   
129  54448.107269 -0.019   0.040 -0.042   0.029    None    None     None   
130  54448.140655  -0.01   0.041  0.051   0.037    None    None     None   
131  54448.207255  0.016   0.042  0.008   0.034    None    None     None   
132  54448.240694  0.016   0.042  0.008   0.034    None    None     None   
133    54448.2742  0.016   0.042  0.008   0.034    None    None     None   

      Uerr        B   Berr       V   Verr  
0    0.797   39.971  1.381  14.393  0.695  

,MJD,UVW2,UVW2err,UVM2,UVM2err,UVW1,UVW1err,U,Uerr,B,Berr,V,Verr
0,54161.85323,0.267,0.048,0.032,0.025,1.199,0.097,15.275,0.797,39.971,1.381,14.393,0.695
1,54165.06481,1.02,0.108,0.106,0.039,5.258,0.337,60.515,2.377,100.605,2.900,32.774,1.213
2,54169.660617,1.279,0.131,0.606,0.084,12.294,0.716,121.711,4.572,183.893,5.006,60.094,1.890
3,54170.63372,2.411,0.226,0.867,0.119,12.538,0.713,121.721,4.573,197.413,5.390,63.164,2.036
4,54171.535133,2.517,0.205,0.686,0.091,13.576,0.756,126.88,4.754,201.98,5.467,67.194,2.057
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,54448.107269,-0.019,0.040,-0.042,0.029,None,None,None,None,-0.29,0.271,-0.056,0.210
130,54448.140655,-0.01,0.041,0.051,0.037,None,None,None,None,0.088,0.284,-0.109,0.208
131,54448.207255,0.016,0.042,0.008,0.034,None,None,None,None,-0.092,0.277,-0.173,0.206
132,54448.240694,0.016,0.042,0.008,0.034,None,None,None,None,-0.092,0.277,-0.173,0.206
